In [4]:
import pandas as pd
summary=pd.read_csv('./Data/Contest1236/standings_statistics',sep='\t')
summary.head(n=10)

,ParticipantID,Rank,ProblemA_id,ProblemA_memory,ProblemA_time,ProblemA_language,ProblemB_id,ProblemB_memory,ProblemB_time,ProblemB_language,ProblemC_id,ProblemC_memory,ProblemC_time,ProblemC_language
0,29090425,1,62793808,100 KB,171 ms,Kotlin,62796367,16 KB,124 ms,Python 3,62791588,8 KB,311 ms,FPC
1,29058912,2,62784582,8 KB,31 ms,GNU C++17,62786209,8 KB,46 ms,GNU C++17,62792622,716 KB,46 ms,GNU C++17
2,29053879,3,62784627,8 KB,31 ms,GNU C++17,62787426,8 KB,31 ms,GNU C++17,62791071,3924 KB,46 ms,GNU C++17
3,29088469,4,62784321,4 KB,31 ms,GNU C++17,62787186,8 KB,31 ms,GNU C++17,62789610,428 KB,46 ms,GNU C++17
4,29087161,5,62785968,8 KB,46 ms,GNU C++14,62788924,8 KB,31 ms,GNU C++14,62791879,1296 KB,46 ms,GNU C++14
5,29089808,6,62783805,8 KB,31 ms,GNU C++11,62792966,8 KB,31 ms,GNU C++11,62789897,12 KB,46 ms,GNU C++11
6,29054986,7,62784292,16 KB,30 ms,GNU C++14,62790776,20 KB,46 ms,GNU C++14,62794126,428 KB,46 ms,GNU C++14
7,29091733,8,62799939,8 KB,31 ms,GNU C++17,62793499,8 KB,31 ms,GNU C++17,62795764,1284 KB,46 ms,GNU C++17
8,29090440,9,62784596,8 KB,31 ms,GNU C++17,62788719,8 KB,31 ms,GNU C++17,62792967,444 KB,46 ms,GNU C++17
9,29074379,10,62785095,8 KB,30 ms,GNU C++11,62790312,8 KB,46 ms,GNU C++11,62790282,8 KB,46 ms,GNU C++11


In [1]:
#collect all code from standings folders into one folder
import os
from shutil import copyfile

for outer_d in os.listdir('./Data'):
    outer_d=os.path.join('./Data',outer_d)
    if os.path.isdir(outer_d):
        #create dir for all codes
        copy_dir=os.path.join(outer_d,"all")
        if not os.path.exists(copy_dir):
            os.makedirs(copy_dir) 
        for d in os.listdir(outer_d):
            d=os.path.join(outer_d,d)
            if os.path.isdir(d) and d!=copy_dir:
                for f in os.listdir(d):
                    file_name=f
                    f=os.path.join(d,f)
                    if os.path.isfile(f):
                        copyfile(f,os.path.join(copy_dir,file_name))

In [6]:
#generate xml for all files using srcml
for outer_d in os.listdir('./Data'):
    outer_d=os.path.join('./Data',outer_d) #Contest1,2,3,...
    if os.path.isdir(outer_d):
        #create dir for xml 
        xml_dir=os.path.join(outer_d,"all_xml")
        if not os.path.exists(xml_dir):
            os.makedirs(xml_dir) 
        for f in os.listdir(os.path.join(outer_d,'all')):
            fname=f
            f=os.path.join(os.path.join(outer_d,'all'),f)
            if os.path.isfile(f):
                cmd="srcml %s -o %s"%(f,os.path.join(xml_dir,fname.replace('.cpp','.xml')))
                os.system(cmd)

In [7]:
import subprocess
proc=subprocess.Popen('srcml --xpath "string(//src:function/src:type)" example.xml', stdout=subprocess.PIPE, shell=True)
(out, err) = proc.communicate()
print("program output:", out)

program output: b'int\n'


In [148]:
import subprocess
from collections import defaultdict
import pandas as pd
import xml.etree.ElementTree as ET

def get_function_names(root):
    fn_names=[]
    for child in root:
        if child.tag.endswith('function'):
            for grandchild in child:
                if grandchild.tag.endswith('name'):
                    fn_names.append(grandchild.text)
    return fn_names

def call_shell(f,filepath):
    cmd=f(filepath)
    proc=subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True)
    (out, err) = proc.communicate()
    return out

def get_features_for_file(filepath):
    features=[]
    feature_generators={'Function_Return_type':(lambda filepath: ('srcml --xpath "string(//src:function/src:type)" %s')%(filepath),"XPATH"), 
                        'Function_Params':(lambda filepath: ('srcml --xpath "string(//src:function/src:parameter_list/src:parameter)" %s'%(filepath)),"XPATH"),
                        'Function_Name': (lambda root: get_function_names(root),"FUNCTION"),
                        'Typedefs':(lambda filepath: ("srcml --xpath '//src:typedef' %s | xpath '//unit/typedef/descendant::text()'"%(filepath)),"XPATH")
                        }
    tree=ET.parse(filepath)
    root=tree.getroot()
    for feature_name,feature_l in feature_generators.items():
        feature_lambda,feature_lambda_type=feature_l
        features.append((feature_name,call_shell(feature_lambda,filepath) if feature_lambda_type=='XPATH' else feature_lambda(root)))
    return features
        
        
def generate_features(folder_path):
    df = defaultdict(lambda: [])
    c=0
    for f in os.listdir(folder_path):
        filename=f
        f=os.path.join(folder_path,f)
        if os.path.isfile(f):
            features=get_features_for_file(f)
            df['filename'].append(filename)
            for feature in features:
                df[feature[0]].append(feature[1])
        c+=1
        if c==100:
            break
    return pd.DataFrame(df)

In [149]:
x=generate_features('./Data/Contest1236/all_xml')
x.to_csv('primitive_features.csv',index=False)

In [131]:
get_features_for_file('./Data/Contest1236/all_xml/29087676_62789478.xml')

[('Function_Return_type', b'long long\n'),
 ('Function_Params', []),
 ('Function_Name', ['quickpow', 'main'])]

In [139]:
import xml.etree.ElementTree as ET
tree = ET.parse('./Data/Contest1236/all_xml/29087676_62789478.xml')
root=tree.getroot()
for fn in root:
    if fn.tag.endswith('function'):
        for param_list in child:
            if param_list.tag.endswith('parameter_list'):
                for gc in param_list:
                    for gcc in gc:
                        print(gcc.tag)
                print(grandchild.attrib)

{}
{}


In [151]:
cmd="srcml --xpath '//src:typedef/src:type' ./example.cpp | xpath '//unit/type/descendant::text()'"
proc=subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True)
(out, err) = proc.communicate()
print(out)

b'long longunsigned long long'
